# Coding Project Analysis JEM Forms


### Data Output: Date, Specimen ID, Rig Opertator, Layer, Cell Type Prediction (Human)

In [1]:
import os
import csv
import json
import fnmatch
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.io.json import json_normalize
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
#from single_cell_ephys.lims_funcs import limsquery
#from single_cell_ephys.file_funcs import validated_input

In [2]:
pd.options.display.max_columns = None  #Displays all the columns 

In [3]:
#Used csv from 5/25/2018
shiny_human_df = pd.read_csv('C:/users/ramr/documents/github/personal-projects/csv/shiny_human.csv')
shiny_mouse_df = pd.read_csv('C:/users/ramr/documents/github/personal-projects/csv/shiny_mouse.csv')

In [4]:
#Still have to fix this and add more definitions for common functions

def gen_fil(df, col, cond):
    '''
    Filters the column with a certain condition
    df: dataframe
    col: column
    cond: condition
    '''
    filtered = df[df[col] == cond]
    return filtered

def gen_cat(df, col):
    '''
    Assigns astype to the column
    df: dataframe
    col: column
    '''    
    category = df[col].astype('category')
    return category

def cell_count(P_num):
    '''
    Counts number of Lims tube id based on P# as number of cells
    df = dataframe
    count() = counts number
    '''
    user = df1[df1['Lims tube id'].str.contains(P_num)]
    return user['Lims tube id'].count()

def P_user(P_num):
    '''
    Prints cell_count 
    '''
    print("Total cells:", cell_count(P_num))

In [5]:
#Post patch class Script by Rusty

def nucleated(x):
    nuc_high_seal = x[(x['extraction.postPatch'] == 'Nucleated') | 
                      (((x['extraction.postPatch'] == 'nucleus_visible') | 
                      (x['extraction.postPatch'] == 'nucleus_present')) & 
                      (x['extraction.endPipetteR'] >= 500))]
    return nuc_high_seal

def partial_nucleated(y):
    nuc_low_seal = y[(y['extraction.postPatch'] == 'Partial-Nucleus') | 
                     (((y['extraction.postPatch'] == 'nucleus_present') | 
                     (y['extraction.postPatch'] == 'nucleus_visible')) & 
                     (y['extraction.endPipetteR'] <= 499))]
    return nuc_low_seal

def outside_out(z):
    no_high_seal = z[(z['extraction.postPatch'] == 'Outside-Out') | 
                     (((z['extraction.postPatch'] == 'nucleus_absent') | 
                     (z['extraction.postPatch'] == 'no_nucleus_visible')) & 
                     (z['extraction.endPipetteR'] >= 500))]
    return no_high_seal

def no_seal(w): 
    no_low_seal = w[(w['extraction.postPatch'] == 'No-Seal') | 
                    (((w['extraction.postPatch'] == 'nucleus_absent') | 
                    (w['extraction.postPatch'] == 'no_nucleus_visible')) & 
                    (w['extraction.endPipetteR'] <= 499))]
    return no_low_seal
    
def entire_cell(v):
    entire = v[(v['extraction.postPatch'] == 'Entire-Cell') | 
               (v['extraction.postPatch'] == 'entire_cell')]
    return entire

#variable['post_patch'] = 'Term'
#Term is an output displayed in the Post_Patch column

def reclassify(df):
    nu = nucleated(df)
    nu['post_patch'] = 'Nuc-high seal' 
    oo = outside_out(df)
    oo['post_patch'] = 'No-high seal'
    pn = partial_nucleated(df)
    pn['post_patch'] = 'Nuc-low seal'
    ns = no_seal(df)
    ns['post_patch'] = 'No-low seal'
    ec = entire_cell(df)
    ec['post_patch'] = 'Entire cell'
    return  nu, oo, pn, ns, ec

def concat_df(a, b, c, d, e):
    frames = (a, b, c, d, e)
    df = pd.concat(frames)
    return df

def postpatch_reclass(df):
    return concat_df(*reclassify(df))

#json_df = postpatch_reclass(json_df)

In [6]:
def get_jsons(dirname, expt):
    json_paths = []
    for jfile in os.listdir(dirname):
        if fnmatch.fnmatch(jfile, '*.%s.json' %expt):
            jpath = os.path.join(dirname, jfile)
            json_paths.append(jpath)
    return json_paths

In [7]:
#make list of json paths for every json file in MIES Experiments directory on 279
json_list = get_jsons("//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/", "PS")
#json_list

In [8]:
def flatten_attempts(slice_info, form_version):
    """Return flattened slice metadata dataframe.
    
    Parameters
    ----------
    slice_info : dict
        A dictionary of slices with nested pipette attempts.
    
    form_version : string
        A string containing the JEM form version.
        (Pre-version 2 contains IVSCC, PatchSeq and Electroporation arrays)
    Returns
    -------
    attempts_slice_df : pandas dataframe
        A dataframe of all pipette attempts along with all slice metadata.
    """
    
    df = json_normalize(slice_info)
    if form_version >= "2":
        ps_array_name = "pipettes"
    else:
        ps_array_name = "pipettesPatchSeqPilot"
    try:
        attempts_df = json_normalize(slice_info[ps_array_name])
        attempts_df["limsSpecName"] = df["limsSpecName"].values[0]
        attempts_df["attempt"] = [p+1 for p in attempts_df.index.values]
        attempts_slice_df = pd.merge(df, attempts_df, how="outer", on="limsSpecName")
        attempts_slice_df.drop(ps_array_name, axis=1, inplace=True)
        return attempts_slice_df
    except IndexError:
        pass

def is_field(df, colname):
    """Determine whether a column name exists in a dataframe.
    
    Parameters
    ----------
    df : a Pandas dataframe
    colname : string
        
    Returns
    -------
    Boolean
        Boolean value indicating if the colname exists in the dataframe.
    """
    
    try:
        df[colname]
        return True
    except KeyError:
        return False

### json_df

In [9]:
json_df = pd.DataFrame()

for json_path in json_list:
    with open(json_path) as data_file:
        slice_info = json.load(data_file)
        if is_field(slice_info, "formVersion"):
            jem_version = slice_info["formVersion"]
        else:
            jem_version = "1.0.0"
        flat_df = flatten_attempts(slice_info, jem_version)
        json_df = pd.concat([json_df, flat_df], axis=0)
        
json_df.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
0,2018-04-18,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,1.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:20:14 -07:00,13:20:13 -07:00,13:20:14 -07:00,NaN,PXS4_180418_551_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:19:59 -07:00,13:20:00 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
1,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,2.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_present,-80.0,-80.0,NaN,NaN,13:21:44 -07:00,13:21:44 -07:00,13:21:45 -07:00,NaN,PXS4_180418_552_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:21:26 -07:00,13:21:31 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
2,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,3.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:22:39 -07:00,13:22:38 -07:00,13:22:39 -07:00,NaN,PXS4_180418_553_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:22:25 -07:00,13:22:27 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
3,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,not_intentionally,nucleus_present,-80.0,-80.0,NaN,NaN,13:23:59 -07:00,13:23:58 -07:00,13:23:59 -07:00,NaN,PXS4_180418_554_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:23:04 -07:00,13:23:09 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
4,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,5.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_absent,-80.0,-80.0,NaN,NaN,13:25:01 -07:00,13:25:01 -07:00,13:25:01 -07:00,NaN,PXS4_180418_555_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:24:37 -07:00,13:24:40 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN


### Date column & include post_patch column

In [10]:
json_df['date'] = json_df['date'].str[:10] #Strip away the time
json_df['date'] = pd.to_datetime(json_df['date']) #Converting to YYYY-MM-DD
json_df.set_index('date', inplace = True)

In [11]:
json_df = postpatch_reclass(json_df) #Makes post patch column

### Creating new dataframe based on date start selection

In [12]:
#['Start date' : 'End date'] & Production Start Date = '2017-10-01' 
#start_date = input('Enter the start date for your dataframe (YYYY-MM-DD): ')
start_date = '2017-10-01'
json_df1 = json_df[start_date:]

json_df1.sort_index(inplace = True)
json_df1 = gen_fil(json_df1, 'status', 'SUCCESS') 

In [13]:
json_df1.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,post_patch
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,None,NaN,NaN,NaN,NaN,3,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,930,NaN,NaN,no,nucleus_present,-26.6,-26.6,No Bubbles,NaN,15:29:19,15:28:58,15:31:31,15:29:19,404,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,15:24:12,15:25:01,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,Nuc-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,30,No alexa fill post experiment,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:10,15:55:26,16:01:35,15:56:11,357,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,15:46:38,15:47:49,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,Cell Dimmed,no,nucleus_absent,-23.2,-23.2,No Bubbles,NaN,15:15:46,15:15:19,15:17:06,15:15:47,403,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5.6,NaN,15:09:53,15:10:41,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,2,3.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,4.7,NaN,NaN,no,nucleus_absent,-26.6,-26.6,NaN,NaN,15:42:11,15:41:40,15:43:57,15:42:11,405,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5.9,NaN,15:37:45,15:38:32,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,5.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,40,NaN,Fluorescence in Pipette,no,nucleus_absent,-35.0,-35.0,No Bubbles,NaN,16:07:20,16:06:38,16:13:41,16:07:21,056,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:59:05,16:01:26,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,No-high seal


### Create a filter for Human & Mouse dataframe

In [14]:
json_df1['human?'] = json_df1['limsSpecName'].str.match(r"H\d\d") #Creates new boolean column

json_df2 = gen_fil(json_df1, 'human?', True) #HUMAN
json_df3 = gen_fil(json_df1, 'human?', False) #MOUSE
json_df1.head()

#json_df2 = json_df1[json_df1['limsSpecName'].str.match('H1')] #Human
#json_df3 = json_df1[json_df1['limsSpecName'] != 'H'] #Mouse

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,post_patch,human?
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,None,NaN,NaN,NaN,NaN,3,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,930,NaN,NaN,no,nucleus_present,-26.6,-26.6,No Bubbles,NaN,15:29:19,15:28:58,15:31:31,15:29:19,404,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,15:24:12,15:25:01,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,30,No alexa fill post experiment,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:10,15:55:26,16:01:35,15:56:11,357,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,15:46:38,15:47:49,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,Cell Dimmed,no,nucleus_absent,-23.2,-23.2,No Bubbles,NaN,15:15:46,15:15:19,15:17:06,15:15:47,403,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5.6,NaN,15:09:53,15:10:41,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,2,3.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,4.7,NaN,NaN,no,nucleus_absent,-26.6,-26.6,NaN,NaN,15:42:11,15:41:40,15:43:57,15:42:11,405,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5.9,NaN,15:37:45,15:38:32,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,5.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,40,NaN,Fluorescence in Pipette,no,nucleus_absent,-35.0,-35.0,No Bubbles,NaN,16:07:20,16:06:38,16:13:41,16:07:21,056,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:59:05,16:01:26,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,No-high seal,False


In [15]:
#Human
json_df2.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,post_patch,human?
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,4,8.0,NaN,"8,16-17,",09/25/2017,NaN,NaN,NaN,200,NaN,NaN,no,nucleus_absent,-52.0,-52.0,NaN,NaN,18:44:31,18:40:55,18:44:33,18:44:32,506,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.04,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,18:29:54,18:30:42,4,dijonh,NaN,NaN,SUCCESS,NaN,Ephys_171009_03_A02_H,No-high seal,True
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,faint alexa fill visible. Might be a plume?,NaN,no,nucleus_absent,-50.0,-50.0,NaN,NaN,20:21:20,20:20:22,20:25:44,20:21:21,360,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.09,NaN,NaN,NaN,NaN,Pyramidal,NaN,4.3,NaN,20:08:19,20:09:12,3,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171009_03_A04_H,No-high seal,True
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 3",NaN,None,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,alexa fill visible post experiment.,NaN,no,nucleus_absent,-50.0,-50.0,No Bubbles,NaN,20:13:41,20:12:54,20:16:52,20:13:41,058,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.10,NaN,NaN,NaN,NaN,Unknown Interneuron,NaN,4.1,NaN,20:02:42,20:04:05,7,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171009_03_A03_H,No-high seal,True
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 3",NaN,None,NaN,NaN,NaN,NaN,2,5.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,5,alexa fill visible post experiment.,NaN,no,nucleus_absent,-60.0,-80.0,Large Bubbles,NaN,20:44:24,20:43:44,20:49:21,20:44:24,059,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.10,NaN,NaN,NaN,NaN,Unknown Interneuron,NaN,3.7,NaN,20:33:08,20:35:46,7,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171009_03_A03_H,No-high seal,True
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 3",NaN,None,NaN,NaN,NaN,NaN,3,7.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10,alexa fill visible post experiment.,NaN,no,nucleus_absent,-30.0,-50.0,No Bubbles,NaN,18:29:42,18:29:04,18:35:07,18:29:42,056,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.10,NaN,NaN,NaN,NaN,NaN,NaN,4.8,NaN,18:19:05,18:20:42,7,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171009_03_A03_H,No-high seal,True


In [16]:
#Mouse
json_df3.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,post_patch,human?
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,None,NaN,NaN,NaN,NaN,3,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,930,NaN,NaN,no,nucleus_present,-26.6,-26.6,No Bubbles,NaN,15:29:19,15:28:58,15:31:31,15:29:19,404,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,15:24:12,15:25:01,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,30,No alexa fill post experiment,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:10,15:55:26,16:01:35,15:56:11,357,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,15:46:38,15:47:49,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,Cell Dimmed,no,nucleus_absent,-23.2,-23.2,No Bubbles,NaN,15:15:46,15:15:19,15:17:06,15:15:47,403,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5.6,NaN,15:09:53,15:10:41,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,2,3.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,4.7,NaN,NaN,no,nucleus_absent,-26.6,-26.6,NaN,NaN,15:42:11,15:41:40,15:43:57,15:42:11,405,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5.9,NaN,15:37:45,15:38:32,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B01_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,5.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,40,NaN,Fluorescence in Pipette,no,nucleus_absent,-35.0,-35.0,No Bubbles,NaN,16:07:20,16:06:38,16:13:41,16:07:21,056,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:59:05,16:01:26,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,No-high seal,False


## HUMAN dataframe

In [17]:
human_df = json_df2[['limsSpecName',
                     'rigOperator',
                     'approach.manualRoi',
                     'manualRoi', 
                     'recording.humanCellTypePrediction',
                     'post_patch',
                     'extraction.tubeID']]

human_df.sort_index(inplace = True)
#approach.manualRoi available data at 2017-10-10 to 2017-11-27
#manualRoi avaible data at 2017-11-27 to Present
#extraction.tubeID changes from ### to User_Date_###_A01 at 2017-11-27

In [18]:
human_df.head()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,recording.humanCellTypePrediction,post_patch,extraction.tubeID
date,,,,,,,
2017-10-10,H17.03.014.11.04,dijonh,"TCx, layer 2/3",NaN,NaN,No-high seal,506
2017-10-10,H17.03.014.11.09,lindsayn,"TCx, layer 5",NaN,Pyramidal,No-high seal,360
2017-10-10,H17.03.014.11.10,rustym,"TCx, layer 3",NaN,Unknown Interneuron,No-high seal,058
2017-10-10,H17.03.014.11.10,rustym,"TCx, layer 3",NaN,Unknown Interneuron,No-high seal,059
2017-10-10,H17.03.014.11.10,rustym,"TCx, layer 3",NaN,NaN,No-high seal,056


In [19]:
human_df.tail()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,recording.humanCellTypePrediction,post_patch,extraction.tubeID
date,,,,,,,
2018-05-22,H18.03.012.11.07.02,rustym,NaN,TCx3,Unknown Interneuron,Nuc-high seal,P2S4_180522_052_A01
2018-05-22,H18.03.012.11.07.02,rustym,NaN,TCx3,FS Interneuron,No-low seal,P2S4_180522_056_A01
2018-05-22,H18.03.012.11.07.02,rustym,NaN,TCx3,Unknown,Nuc-high seal,P2S4_180522_054_A01
2018-05-22,H18.03.012.11.13.01,ramr,NaN,TCx2,Unknown Interneuron,No-low seal,PAS4_180522_451_A01
2018-05-22,H18.03.012.11.07.02,rustym,NaN,TCx3,Pyramidal,Nuc-high seal,P2S4_180522_053_A01


In [20]:
shiny_initial = ['sample_id',
                 'cell_name_label',
                 'res_index_label']
shiny_human_df = shiny_human_df[shiny_initial]
shiny_human_df.head()

,sample_id,cell_name_label,res_index_label
0,P2S4_170323_053_A01,H17.06.007.11.04.01,1.0
1,P2S4_170323_054_A01,H17.06.007.11.04.02,1.0
2,P2S4_170323_055_A01,H17.06.007.11.04.03,1.0
3,P2S4_170323_056_A01,H17.06.007.11.04.04,1.0
4,P5S4_170323_201_A01,H17.06.007.11.09.01,1.0


In [21]:
final_H_df = pd.merge(left = human_df, 
                    right = shiny_human_df, 
                    left_on = 'extraction.tubeID', 
                    right_on = 'sample_id', 
                    how = 'inner')

In [22]:
final_H_df.head()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,recording.humanCellTypePrediction,post_patch,extraction.tubeID,sample_id,cell_name_label,res_index_label
0,H17.26.003.11.16,lindsayn,NaN,TCx2,Unknown Interneuron,Nuc-high seal,P8S4_171127_352_A01,P8S4_171127_352_A01,H17.26.003.11.16.02,0.832207
1,H17.26.003.11.16,lindsayn,NaN,TCx2,Unknown Interneuron,Nuc-high seal,P8S4_171127_354_A01,P8S4_171127_354_A01,H17.26.003.11.16.04,1.000000
2,H17.26.003.11.14,dijonh,NaN,TCx3,NaN,Nuc-high seal,PBS4_171127_501_A01,PBS4_171127_501_A01,H17.26.003.11.14.01,1.000000
3,H17.26.003.11.16,lindsayn,NaN,TCx5,Unknown Interneuron,No-high seal,P8S4_171127_353_A01,P8S4_171127_353_A01,H17.26.003.11.16.03,0.504463
4,H17.26.003.11.14,dijonh,NaN,TCx5,NaN,Nuc-high seal,PBS4_171127_502_A01,PBS4_171127_502_A01,H17.26.003.11.14.02,1.000000


In [23]:
final_H_df.tail()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,recording.humanCellTypePrediction,post_patch,extraction.tubeID,sample_id,cell_name_label,res_index_label
225,H18.03.005.11.11,lisak,NaN,TCx3,FS Interneuron,No-high seal,P9S4_180320_409_A01,P9S4_180320_409_A01,H18.03.005.11.11.02,0.254718
226,H18.03.005.11.04,ramr,NaN,TCx2,Unknown Interneuron,No-low seal,PAS4_180321_451_A01,PAS4_180321_451_A01,H18.03.005.11.04.01,1.000000
227,H18.03.007.11.05,rustym,NaN,TCx3,Pyramidal,Nuc-high seal,P2S4_180327_055_A01,P2S4_180327_055_A01,H18.03.007.11.05.01,1.000000
228,H18.03.007.11.05,rustym,NaN,TCx2,Pyramidal,Nuc-high seal,P2S4_180327_056_A01,P2S4_180327_056_A01,H18.03.007.11.05.02,1.000000
229,H18.03.007.11.05,rustym,NaN,TCx2,Pyramidal,No-high seal,P2S4_180327_057_A01,P2S4_180327_057_A01,H18.03.007.11.05.03,0.365141


In [24]:
human_df['post_patch'].value_counts()

Nuc-high seal    183
No-low seal      123
No-high seal     108
Nuc-low seal      49
Entire cell        3
Name: post_patch, dtype: int64

In [25]:
final_H_df['post_patch'].value_counts()

Nuc-high seal    79
No-low seal      65
No-high seal     59
Nuc-low seal     25
Entire cell       2
Name: post_patch, dtype: int64

## MOUSE dataframe

In [26]:
mouse_df = json_df3[['limsSpecName',
                     'rigOperator',
                     'approach.manualRoi',
                     'manualRoi', 
                     'post_patch',
                     'extraction.tubeID']]

mouse_df.sort_index(inplace = True)
#approach.manualRoi available data at 2017-10-02 to 2017-11-10
#manualRoi avaible data at 2017-11-10 to Present
#extraction.tubeID changes from ### to User_Date_###_A01 at 2017-11-10

In [27]:
mouse_df.head()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,post_patch,extraction.tubeID
date,,,,,,
2017-10-02,Gad2-IRES-Cre;Ai14-350672.04.01,lisak,"VISp, layer 4",NaN,Nuc-high seal,404
2017-10-02,Chrna2-Cre_OE25;Ai14-351067.03.02,lindsayn,"VISp, layer 5",NaN,No-high seal,357
2017-10-02,Gad2-IRES-Cre;Ai14-350672.04.01,lisak,"VISp, layer 2/3",NaN,No-high seal,403
2017-10-02,Gad2-IRES-Cre;Ai14-350672.04.01,lisak,"VISp, layer 2/3",NaN,No-high seal,405
2017-10-02,Chrna2-Cre_OE25;Ai14-351067.04.01,rustym,"VISp, layer 5",NaN,No-high seal,056


In [28]:
mouse_df.tail()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,post_patch,extraction.tubeID
date,,,,,,
2018-05-24,Sst-IRES-Cre;Ai14-391339.03.02,lindsayn,NaN,VISp5,Nuc-high seal,P8S4_180524_357_A01
2018-05-24,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-391574.03.01,dijonh,NaN,VISp4,Nuc-high seal,PBS4_180524_502_A01
2018-05-24,Htr3a-Cre_NO152;Ai14-390983.04.01,dijonh,NaN,VISp4,Nuc-low seal,PBS4_180524_505_A01
2018-05-24,Htr3a-Cre_NO152;Ai14-390983.04.01,dijonh,NaN,VISp2/3,Nuc-high seal,PBS4_180524_503_A01
2018-05-24,Htr3a-Cre_NO152;Ai14-390983.04.02,lindsayn,NaN,VISp4,Nuc-high seal,P8S4_180524_352_A01


In [29]:
shiny_mouse_df = shiny_mouse_df[shiny_initial]
shiny_mouse_df.head()

,sample_id,cell_name_label,res_index_label
0,P1S4_160318_069_A01,Ndnf-IRES2-dgCre;Ai14-234626.02.L.01,1.000000
1,P1S4_160407_108_A01,Ndnf-IRES2-dgCre;Ai14-237573.07.R.01,1.000000
2,P1S4_160407_109_A01,Ndnf-IRES2-dgCre;Ai14-237573.07.L.01,1.000000
3,P1S4_160407_112_A01,Ndnf-IRES2-dgCre;Ai14-237573.07.L.02,1.000000
4,P1S4_160407_113_A01,Ndnf-IRES2-dgCre;Ai14-237573.07.L.03,0.802691


In [30]:
final_M_df = pd.merge(left = mouse_df, 
                    right = shiny_mouse_df, 
                    left_on = 'extraction.tubeID', 
                    right_on = 'sample_id', 
                    how = 'inner')

In [31]:
final_M_df.head()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,post_patch,extraction.tubeID,sample_id,cell_name_label,res_index_label
0,Pvalb-IRES-Cre;Ai14-357946.02.01,aarono,NaN,VISp2/3,No-high seal,PXS4_171108_251_A01,PXS4_171108_251_A01,Pvalb-IRES-Cre;Ai14-357946.02.01.01,0.458638
1,Sst-IRES-Cre;Ai14-357932.03.02,lindsayn,NaN,VISp5,Nuc-high seal,P8S4_171110_355_A01,P8S4_171110_355_A01,Sst-IRES-Cre;Ai14-357932.03.02.02,0.703208
2,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-357672.04.01,lindsayn,NaN,VISp5,Nuc-high seal,P8S4_171110_353_A01,P8S4_171110_353_A01,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-357672.04.01.01,1.000000
3,Nkx2-1-CreERT2;Ai14-357183.06.02,ramr,NaN,VISp2/3,No-high seal,PAS4_171110_451_A01,PAS4_171110_451_A01,Nkx2-1-CreERT2;Ai14-357183.06.02.01,0.329975
4,Nkx2-1-CreERT2;Ai14-357183.04.02,lindsayn,NaN,VISp5,No-high seal,P8S4_171110_351_A01,P8S4_171110_351_A01,Nkx2-1-CreERT2;Ai14-357183.04.02.01,0.329975


In [32]:
final_M_df.tail()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,post_patch,extraction.tubeID,sample_id,cell_name_label,res_index_label
1318,Rbp4-Cre_KL100;Ai14-383389.03.01,dijonh,NaN,VISp4,Nuc-low seal,PBS4_180409_506_A01,PBS4_180409_506_A01,Rbp4-Cre_KL100;Ai14-383389.03.01.02,1.000000
1319,Slc32a1-IRES-Cre;Ai14-384210.03.01,kristenh,NaN,VISp2/3,No-high seal,P1S4_180410_003_A01,P1S4_180410_003_A01,Slc32a1-IRES-Cre;Ai14-384210.03.01.01,0.688578
1320,Vip-IRES-Cre;Ai14-383085.05.01,kristenh,NaN,VISp2/3,Entire cell,P1S4_180410_001_A01,P1S4_180410_001_A01,Vip-IRES-Cre;Ai14-383085.05.01.01,1.000000
1321,Vip-IRES-Cre;Ai14-383085.05.01,kristenh,NaN,VISp2/3,No-low seal,P1S4_180410_002_A01,P1S4_180410_002_A01,Vip-IRES-Cre;Ai14-383085.05.01.02,0.458638
1322,Vip-IRES-Cre;Ai14-383085.03.02,rustym,NaN,VISp2/3,No-high seal,P2S4_180410_051_A01,P2S4_180410_051_A01,Vip-IRES-Cre;Ai14-383085.03.02.01,0.562351


In [33]:
mouse_df['post_patch'].value_counts()

Nuc-high seal    1373
No-high seal      527
No-low seal       435
Nuc-low seal      284
Entire cell        40
Name: post_patch, dtype: int64

In [34]:
final_M_df['post_patch'].value_counts()

Nuc-high seal    649
No-low seal      257
No-high seal     228
Nuc-low seal     171
Entire cell       18
Name: post_patch, dtype: int64

## Extra

In [35]:
def nuc_count(df):
    #user = str(input('Enter the user number (P#): '))
    user = 'PA'
    
    nh = df[df['post_patch'].str.contains('Nuc-high seal') & 
            df['extraction.tubeID'].str.contains(user)]
    nh = nh['post_patch'].count()

    nl = df[df['post_patch'].str.contains('Nuc-low seal') & 
            df['extraction.tubeID'].str.contains(user)]
    nl = nl['post_patch'].count()

    noh = df[df['post_patch'].str.contains('No-high seal') & 
             df['extraction.tubeID'].str.contains(user)]
    noh = noh['post_patch'].count()

    nol = df[df['post_patch'].str.contains('No-low seal') & 
             df['extraction.tubeID'].str.contains(user)]
    nol = nol['post_patch'].count()

    en = df[df['post_patch'].str.contains('Entire cell') & 
            df['extraction.tubeID'].str.contains(user)]
    en = en['post_patch'].count()
    

    print 'Nucleated:', nh
    print 'Partial nucleated', nl
    print 'Outside-out:', noh
    print 'No seal:', nol
    print 'Entire cell:', en   

In [36]:
nuc_count(mouse_df)

Nucleated: 134
Partial nucleated 59
Outside-out: 8
No seal: 41
Entire cell: 7


In [37]:
nuc_count(final_M_df)

Nucleated: 93
Partial nucleated 37
Outside-out: 7
No seal: 26
Entire cell: 7


In [38]:
nuc_count(human_df)

Nucleated: 2
Partial nucleated 4
Outside-out: 2
No seal: 10
Entire cell: 0


In [39]:
nuc_count(final_H_df)

Nucleated: 2
Partial nucleated 2
Outside-out: 2
No seal: 2
Entire cell: 0
